In [1]:
import pandas as pd
df = pd.read_csv("GBP_JPY_D_2005_202206.csv")
df.columns=['datetime', 'open', 'high', 'low', 'close']
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
df.head(10)

,datetime,open,high,low,close
0,2005-01-01 22:00:00,196.780,197.300,196.75,197.150
1,2005-01-02 22:00:00,197.140,197.150,195.40,195.750
2,2005-01-03 22:00:00,195.760,197.080,195.12,196.870
3,2005-01-04 22:00:00,196.830,197.040,195.61,196.070
4,2005-01-05 22:00:00,196.050,197.300,195.85,196.930
5,2005-01-06 22:00:00,196.930,197.060,195.43,196.000
6,2005-01-07 22:00:00,196.000,196.000,196.00,196.000
7,2005-01-08 22:00:00,195.990,196.215,195.89,196.055
8,2005-01-09 22:00:00,196.065,196.345,195.10,195.685
9,2005-01-10 22:00:00,195.675,195.815,193.66,194.045


In [2]:
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from zigzag import *

/Applications/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [109]:
X = df['close'] 
pivots = peak_valley_pivots(X.values, 0.05, -0.05)
ts_pivots = pd.Series(X, index=X.index)
ts_pivots = ts_pivots[pivots != 0]

In [110]:
import plotly.graph_objects as go
from datetime import datetime


fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
fig.show()
fig.add_trace(go.Scatter(x=ts_pivots.index, y=ts_pivots.values,line_color='blue'))

fig.show()


In [111]:
import warnings
warnings.filterwarnings('ignore')

channel_up = np.array([0]*len(df))
channel_down = np.array([0]*len(df))
u=0 #number of cahnnel up
d=0 #numbe of channel bottom
w=3
from scipy.signal import argrelextrema
for i in range(len(ts_pivots.index)-1):
    
    s=ts_pivots.index[i]
    e=ts_pivots.index[i+1]

    temp = df[s:e]

    local_min = np.array([])
    local_max = np.array([])
    max_idx = np.array([])
    min_idx = np.array([])

    for i in range(0,len(temp),w):
        local_min = np.append(local_min, temp.low.iloc[i:i+w].min())
        min_idx = np.append(min_idx, temp.low.iloc[i:i+w].idxmin())
    for i in range(0,len(temp),w):
        local_max = np.append(local_max, temp.high.iloc[i:i+w].max())
        max_idx = np.append(max_idx, temp.high.iloc[i:i+w].idxmax())

    k_min, b_min = np.polyfit(min_idx, local_min,1)
    k_max, b_max = np.polyfit(max_idx, local_max,1)

    if k_min>0 and k_max>0:
        if k_min/k_max <1.1 and k_min/k_max>0.9:
            u+=1
            channel_up[s:e]=1

            fig = go.Figure(data=[go.Candlestick(x=temp.index,
                            open=temp['open'],
                            high=temp['high'],
                            low=temp['low'],
                            close=temp['close'])])
            d_max = (df.high.iloc[max_idx] - k_max*max_idx).max()
            d_min = (df.low.iloc[min_idx] - k_min*min_idx).min()
            fig.add_trace(go.Scatter(x=min_idx, y=k_min*min_idx + d_min, mode='lines', name='lower line'))
            fig.add_trace(go.Scatter(x=max_idx, y=k_max*max_idx + d_max, mode='lines', name='upper line'))

            fig.show()
    elif k_min<0 and k_max<0:
        if k_min/k_max <1.1 and k_min/k_max>0.9:
            d+=1
            channel_down[s:e]=1

            fig = go.Figure(data=[go.Candlestick(x=temp.index,
                            open=temp['open'],
                            high=temp['high'],
                            low=temp['low'],
                            close=temp['close'])])
            d_max = (df.high.iloc[max_idx] - k_max*max_idx).max()
            d_min = (df.low.iloc[min_idx] - k_min*min_idx).min()
            fig.add_trace(go.Scatter(x=min_idx, y=k_min*min_idx + d_min, mode='lines', name='lower line'))
            fig.add_trace(go.Scatter(x=max_idx, y=k_max*max_idx + d_max, mode='lines', name='upper line'))

            fig.show()
    

In [112]:
df['channel_up']=channel_up
df['channel_down']=channel_down

In [115]:
u

23

In [116]:
d

17